In [ ]:
import os
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchmetrics.classification import MultilabelAccuracy
from torchmetrics.classification import MultilabelAUROC
import pytorch_lightning as pl
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F
sys.path.append('../..')
from multi_modal_heart.model.ecg_net_attention import ECGEncoder,ECGAttentionAE
from multi_modal_heart.model.ecg_net import ECGAE
from multi_modal_heart.model.ecg_net import BenchmarkClassifier
from multi_modal_heart.ECG.ecg_dataset import ECGDataset
from torchmetrics import Accuracy
import torch.nn as nn
from lifelines.utils import concordance_index

## set os environment CUBLAS_WORKSPACE_CONFIG=:4096:8
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"


## define survival loss function here
def _negative_log_likelihood(E, risk): 
    '''
    E: 1 if uncensored, 0 if censored
    risk: predicted risk

    ''' 
    hazard_ratio = torch.exp(risk)
    log_risk = torch.log(torch.cumsum(hazard_ratio,dim=0))
    uncensored_likelihood = risk - log_risk
    censored_likelihood = uncensored_likelihood * E
    neg_likelihood = -torch.sum(censored_likelihood)
    return neg_likelihood

class LitSurvivalModel(pl.LightningModule):
    def __init__(self,encoder,input_dim,lr=-4,alpha = 0.5,wd=-5,dropout=0.5,freeze_encoder=False, decoder=None):
        super().__init__()
    
        self.save_hyperparameters(ignore=["encoder", "decoder","freeze_encoder"])
        self.encoder = encoder
        self.decoder = decoder
        self.dropout_input = nn.Dropout(p=dropout)
        self.freeze_encoder = freeze_encoder
        if self.freeze_encoder:
            self.encoder.eval()
            for param in self.encoder.parameters():
                param.requires_grad = False
        #### add a branch for survival prediction regression
        self.downsteam_net = BenchmarkClassifier(input_size=input_dim,hidden_size=128,output_size=1)

        self.test_score_list = []
        self.y_status_list = []
        self.y_duration_list = []
        self.c_index = 0
    def forward(self, x, mask=None):
        latent_code = self.encoder.get_features_after_pooling(x,mask)
        # print (latent_code.shape)
        self.latent_code  = latent_code
        log_risk_score = self.downsteam_net(latent_code)

        if self.decoder is not None:
            recon_signal = self.decoder(latent_code)
            return [log_risk_score,recon_signal]
        return [log_risk_score]

    def run_batch(self, batch, batch_idx, stage="train"):
         ## sort the batch by the survival duration of the event
        x = batch[0] ## input signal
        y = batch[1][0] ## censoring status
        duration = batch[1][1] ## survival duration (days)
        ## sort the batch by the survival duration of the event
        duration, sort_idx = duration.sort(descending=True)
        x = x[sort_idx]
        y = y[sort_idx]

        x = self.dropout_input(x)
        if self.decoder is not None:
            y_hat,recon_signal = self(x,mask=None)
            ## survival loss
            surv_loss = _negative_log_likelihood(y,y_hat)
            self.log(f"{stage}/survival_loss", surv_loss,prog_bar=True)

            ## mse loss 
            mse_loss = F.mse_loss(recon_signal, x)
            self.log(f"{stage}/mse_loss", mse_loss,prog_bar=True)
            loss = self.hparams.alpha*surv_loss + (1-self.hparams.alpha)*mse_loss
        else:
            y_hat = self(x,mask=None)
            ## survival loss
            loss = _negative_log_likelihood(y, y_hat)
            self.log(f"{stage}/survival_loss", loss,prog_bar=True)
        
        if "test" in stage:
            # print(stage)
            # print("save risk score")
            self.test_score_list.append(y_hat)
            self.y_status_list.append(y)
            self.y_duration_list.append(duration)
            ## save risk score
        self.log(f"{stage}_loss",loss,prog_bar=True)
        return loss

    
    def training_step(self, batch, batch_idx):
        loss = self.run_batch(batch,batch_idx,stage="train")
        return loss
       
    def test_step(self, batch, batch_idx):
        loss = self.run_batch(batch,batch_idx,stage="test")
        return loss
    
    def on_test_epoch_end(self):
        if (not len(self.test_score_list)==0) and (not len(self.y_status_list)==0) and (not len(self.y_duration_list)==0):
            pred_score = torch.cat(self.test_score_list,dim=0)
            y_status = torch.cat(self.y_status_list,dim=0)
            y_duration = torch.cat(self.y_duration_list,dim=0)
            ## calculate the concordance index
            c_index = concordance_index(y_duration.cpu().numpy(), -pred_score.detach().cpu().numpy(), y_status.cpu().numpy())
            self.log("c_index",c_index)
            self.c_index = c_index
    
    def configure_optimizers(self):
        return torch.optim.AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=self.hparams.lr, weight_decay=self.hparams.wd)

In [2]:
## load MI data without HF before.
import numpy as np
from scipy.stats import zscore
time_steps =608

## save to the numpy array
# np.save("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/batched_ecg_median_wave_MI_with_HF_event.npy", stacked_ecg_data_list)
# np.save("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/y_list_MI_with_HF_event.npy", y_list)
# np.save("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/duration_list_MI_with_HF_event.npy", duration_list)
# np.save("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/status_list_MI_with_HF_event.npy", status_list)

## read the data from the numpy array
stacked_ecg_data_list = np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/batched_ecg_median_wave_MI_with_HF_event.npy")
duration_list = np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/duration_list_MI_with_HF_event.npy")
status_list = np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/status_list_MI_with_HF_event.npy")
eid_list = np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/survival_regression/data/e_id_list_MI_with_HF_event.npy")
time_steps  = 608

stacked_ecg_data_list = zscore(stacked_ecg_data_list,axis=-1)
stacked_ecg_data_list = np.nan_to_num(stacked_ecg_data_list)
pad_num = (time_steps-stacked_ecg_data_list.shape[-1])//2
x = np.pad(stacked_ecg_data_list,((0,0),(0,0),(pad_num,pad_num)),"constant",constant_values=0)
print("input ecg shape",x.shape)

## concatentate the status and duration and eid to the y
y = np.stack([status_list,duration_list,eid_list],axis=-1)
print("status, duration, eid", y.shape)

input ecg shape (919, 12, 608)
status, duration, eid (919, 3)


In [3]:

## split the data into train validate and test, 40% for train, 10% for validate, 50% for test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)
## tenrin to torch tensor dataset
from torch.utils.data import TensorDataset, DataLoader
batch_size = 100
train_dataset = TensorDataset(torch.from_numpy(X_train).float(),torch.from_numpy(y_train).long())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(),torch.from_numpy(y_test).long())
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True) # create your dataloader
test_data_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False) # create your dataloader

print ('num of training data:{}, HF censored event numbers:{}'.format(X_train.shape[0],y_train[:,0].sum()))
print ('num of test data:{}, HF censored event numbers:{}'.format(X_test.shape[0],y_test[:,0].sum()))


num of training data:459, HF censored event numbers:20
num of test data:460, HF censored event numbers:25


In [4]:
import torch
import sys
sys.path.append('../../')
from multi_modal_heart.model.ecg_net import ECGAE
import torch.nn as nn
use_median_wave = True
time_steps = 608
pl.seed_everything(42)
model_name = "resnet1d101_512_pretrained_recon+ECG2Text"

if model_name=="resnet1d101_512_pretrained_recon":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="resnet1d101_512_pretrained_recon+ECG2Text":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="resnet1d101_512_pretrained_classification":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median_finetune/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="resnet1d101_512_pretrained_classification+ECG2Text":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median_finetune/resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="ECG_attention_pretrained_on_classification":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False)
    checkpoint_path = "../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_pretrained_on_classification+ECG2Text":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False)
    checkpoint_path = "../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="ECG_attention_pretrained_on_recon":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_pretrained_on_recon_ECG2Text":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

else:
    raise NotImplementedError


num_classes = 2 ## for binary classification
freeze_encoder = False
train_from_scratch = True
survival_model = LitSurvivalModel(encoder=ecg_net.encoder,
                                  input_dim=512,
                                  decoder = ecg_net.decoder,
                                  lr=1e-3,
                                  alpha = 0.5,
                                  wd=1e-4,
                                  freeze_encoder=freeze_encoder
                                 )
checkpoint = torch.load(checkpoint_path)["state_dict"]
encoder = ecg_net.encoder
decoder = ecg_net.decoder
if not train_from_scratch:
    try:
        encoder_params = {(".").join(key.split(".")[2:]):value for key, value in checkpoint.items() if str(key).startswith("network.encoder")}
        survival_model.encoder.load_state_dict(encoder_params)
    except:
        encoder_params = {(".").join(key.split(".")[1:]):value for key, value in checkpoint.items() if str(key).startswith("encoder")}
        survival_model.encoder.load_state_dict(encoder_params)
    max_epochs = 50
else:
    max_epochs = 300

Global seed set to 42


19
[16, 64, 64, 64, 64]
number of blocks 4


In [5]:
import torch
import sys
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import os
tb_logger = TensorBoardLogger( f"./train_survival_net", name=model_name, version="")  
checkpoint_dir  = os.path.join(tb_logger.log_dir,"checkpoints")

checkpoint_callback_best_loss_min = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, 
                                                    filename='checkpoint_best_train_loss',
                                                    save_top_k=1, monitor="train_loss"
                                                    , mode='min',save_last=True)


callbacks=[
    # FineTuneLearningRateFinder(milestones=[5, 10],min_lr=1e-5, max_lr=1e-3, 
    #                             mode='linear', early_stop_threshold=4.0),
    checkpoint_callback_best_loss_min
    ]


trainer = pl.Trainer(accelerator="gpu",
                    devices=1, max_epochs=2,
                    logger=tb_logger,log_every_n_steps=1,
                    callbacks=callbacks,
                    )
trainer.fit(survival_model, train_dataloaders=train_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/engs2522/local/conda/envs/pytorch3d/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/train_survival_net/resnet1d101_512_pretrained_recon+ECG2Text/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | ECG_ResNetencoder   | 2.1 M 
1 | decoder       | ECG_decoder         | 5.1 M 
2 | dropout_input | Dropout             | 0     
3 | downsteam_net | BenchmarkClassifier | 67.1 K
------------------------------------------------------
7.2 M     Trainable params
0         Non-trainable 

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [149]:
## test the model
survival_model.eval()
trainer.test(survival_model,test_data_loader)
## get the c-index
print ("c-index",survival_model.c_index)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          c_index          │    0.37777777777777777    │
│       test/mse_loss       │    1.6764867305755615     │
│    test/survival_loss     │         9477250.0         │
│         test_loss         │         4738626.0         │
└───────────────────────────┴───────────────────────────┘

c-index 0.37777777777777777


In [8]:
import torch
import sys
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import os
def DL_single_run(train_dataloader,lr=1e-3,alpha = 0.5,wd=1e-5,dropout=0.5,max_epochs=50):
    global encoder
    global decoder
    # pl.seed_everything(42,workers=True)
    # torch.use_deterministic_algorithms(True)
    survival_model = LitSurvivalModel(encoder=encoder,
                                      input_dim=512,
                                      decoder = decoder,
                                      lr=lr,
                                      alpha = alpha,
                                      wd=wd,
                                      dropout = dropout,
                                      freeze_encoder=freeze_encoder
                                     )

    trainer = pl.Trainer(accelerator="gpu",
                        devices=1, max_epochs=max_epochs,
                        )
    trainer.fit(survival_model, train_dataloaders=train_dataloader)
    return trainer,survival_model

In [9]:
from lifelines.utils import concordance_index
import optunity
def hypersearch_DL(x_data, y_data, method, nfolds, nevals, lrexp_range, wdexp_range, dro_range, 
                   alpha_range, num_epochs=300,batch_size=100):

    @optunity.cross_validated(x=x_data, y=y_data, num_folds=nfolds)
    def modelrun(x_train, y_train, x_test, y_test, lrexp, wdexp, dro,alpha):
                ## tenrin to torch tensor dataset
        train_dataset = TensorDataset(torch.from_numpy(x_train).float(),torch.from_numpy(y_train).long())
        test_dataset = TensorDataset(torch.from_numpy(x_test).float(),torch.from_numpy(y_test).long())
        train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True) # create your dataloader
        test_data_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False) # create your dataloader
        trainer,survival_model = DL_single_run(train_dataloader,lr=10**lrexp,
                               alpha = alpha,wd=10**wdexp,dropout=dro,max_epochs=num_epochs)
        
        survival_model.eval()
        trainer.test(survival_model,test_data_loader)
        ## get the c-index
        c_index = survival_model.c_index
        print ("c-index",c_index)
        del survival_model
        del trainer
        return c_index

    optimal_pars, searchlog, _ = optunity.maximize(modelrun, num_evals=nevals, solver_name=method, lrexp=lrexp_range, 
                                                   wdexp=wdexp_range, dro=dro_range,alpha=alpha_range)

    print('Optimal hyperparameters : ' + str(optimal_pars))
    print('Cross-validated C after tuning: %1.3f' % searchlog.optimum)
    return optimal_pars, searchlog 

In [10]:
opars, clog = hypersearch_DL(x_data=x, y_data=y, 
                             method='particle swarm', nfolds=6, nevals=50,
                             lrexp_range=[-6.,-4.5], wdexp_range=[-7,-4], dro_range=[.1,.9], 
                             alpha_range=[0.1, 0.2],
                             batch_size=16, num_epochs=2)

`Trainer.fit` stopped: `max_epochs=2` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          c_index          │    0.5473684210526316     │
│       test/mse_loss       │    1.0331733226776123     │
│    test/survival_loss     │         8240417.5         │
│         test_loss         │        1277442.625        │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | ECG_ResNetencoder   | 2.1 M 
1 | decoder       | ECG_decoder         | 5.1 M 
2 | dropout_input | Dropout             | 0     
3 | downsteam_net | BenchmarkClassifier | 67.1 K
------------------------------------------------------
7.2 M     Trainable params
0         Non-trainable params
7.2 M     Total params
28.958    Total estimated model params size (MB)


...

`Trainer.fit` stopped: `max_epochs=2` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score
test
save risk score


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          c_index          │    0.5342465753424658     │
│       test/mse_loss       │    1.1598665714263916     │
│    test/survival_loss     │         7065011.5         │
│         test_loss         │        1212224.125        │
└───────────────────────────┴───────────────────────────┘

c-index 0.5342465753424658
Optimal hyperparameters : {'lrexp': -5.556884765625, 'wdexp': -6.28369140625, 'dro': 0.44804687500000007, 'alpha': 0.15327148437500002}
Cross-validated C after tuning: 0.584


In [14]:
## retrain the model with the optimal parameters
train_dataset = TensorDataset(torch.from_numpy(X_train).float(),torch.from_numpy(y_train).long())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(),torch.from_numpy(y_test).long())
train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle=True) # create your dataloader
test_data_loader = DataLoader(test_dataset,batch_size=16,shuffle=False) # create your dataloader
trainer,survival_model = DL_single_run(train_dataloader,lr=10**opars["lrexp"],
                            alpha = opars["alpha"],
                            wd=10**opars["wdexp"],
                            dropout=opars["dro"],
                            max_epochs=300)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | ECG_ResNetencoder   | 2.1 M 
1 | decoder       | ECG_decoder         | 5.1 M 
2 | dropout_input | Dropout             | 0     
3 | downsteam_net | BenchmarkClassifier | 67.1 K
------------------------------------------------------
7.2 M     Trainable params
0         Non-trainable params
7.2 M     Total params
28.958    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.


In [13]:
opars

{'lrexp': -5.556884765625,
 'wdexp': -6.28369140625,
 'dro': 0.44804687500000007,
 'alpha': 0.15327148437500002}

In [15]:
## test the model
survival_model.eval()
trainer.test(survival_model,test_data_loader)
## get the c-index
print ("c-index",survival_model.c_index)
## save the model
torch.save(survival_model.state_dict(), f"./train_survival_net/{model_name}/best_model_c_index{str(np.round(survival_model.c_index))}.pth")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          c_index          │    0.47926509186351707    │
│       test/mse_loss       │    1.0882197618484497     │
│    test/survival_loss     │         6980621.0         │
│         test_loss         │         1069931.0         │
└───────────────────────────┴───────────────────────────┘

c-index 0.47926509186351707
